In [5]:
import numpy as np
import pandas as pd
from scipy.stats import t

In [6]:
def kalman_filter_1d_variable_R(measurements, Rvals, Q):
    """
    Applies a 1D Kalman filter with variable measurement noise for each step.
    
    Parameters:
        measurements (np.array): 1D array of measured RSSI values.
        Rvals (np.array): 1D array of measurement noise variances.
        Q (float): Process noise variance.
    
    Returns:
        np.array: Kalman-filtered estimates.
    """
    N = len(measurements)
    xKF = np.zeros(N)
    if N == 0:
        return xKF
    xEst = measurements[0]
    P = 10  # initial covariance (tune as needed)
    xKF[0] = xEst
    for k in range(1, N):
        # PREDICTION
        xPred = xEst  # constant-state model
        PPred = P + Q  # add process noise
        
        # UPDATE
        z = measurements[k]
        Rk = Rvals[k]
        K = PPred / (PPred + Rk)  # Kalman gain
        xEst = xPred + K * (z - xPred)
        P = (1 - K) * PPred
        xKF[k] = xEst
    return xKF

def gesd_outliers(data, threshold_factor=0.5, alpha=0.05):
    """
    Identify outliers in data using an approach similar to the GESD method.
    
    Parameters:
        data (np.array): 1D array of data values.
        threshold_factor (float): Scaling factor for the threshold.
        alpha (float): Significance level.
    
    Returns:
        np.array: Boolean array where True indicates an outlier.
    """
    data = np.array(data)
    n = len(data)
    if n < 3:
        return np.zeros(n, dtype=bool)
    
    outlier_mask = np.zeros(n, dtype=bool)
    indices = np.arange(n)
    remaining = indices.copy()
    
    while len(remaining) >= 3:
        current_data = data[remaining]
        mu = np.nanmean(current_data)
        sigma = np.nanstd(current_data, ddof=1)
        if sigma == 0:
            break
        deviations = np.abs(current_data - mu)
        max_idx_local = np.argmax(deviations)
        R_i = deviations[max_idx_local] / sigma
        n_i = len(current_data)
        p = 1 - alpha / (2 * n_i)
        t_value = t.ppf(p, n_i - 2)
        lambda_i = ((n_i - 1) * t_value) / (np.sqrt((n_i - 2 + t_value**2) * n_i))
        if R_i > threshold_factor * lambda_i:
            outlier_index = remaining[max_idx_local]
            outlier_mask[outlier_index] = True
            remaining = np.delete(remaining, max_idx_local)
        else:
            break
    return outlier_mask

def filter_rssi(master_data, Q=0.01, window_size=25):
    """
    Processes and filters RSSI values for each unique test distance.
    
    Steps performed for each test distance:
        1. GESD outlier detection: mark outliers as NaN.
        2. Linear interpolation of missing values.
        3. Construct a variable R array (measurement noise grows with |RSSI|).
        4. Apply the variable-R Kalman filter.
        5. Apply rolling median smoothing.
    
    Parameters:
        master_data (pd.DataFrame): DataFrame with columns 'TestDistance_m' and 'RSSI'.
        Q (float): Process noise variance for the Kalman filter.
        window_size (int): Window size for the rolling median.
    
    Returns:
        pd.Series: A Series containing the filtered RSSI values.
    """
    cleaned_RSSI = pd.Series(np.nan, index=master_data.index)
    unique_tests = master_data['TestDistance_m'].unique()
    
    for test in unique_tests:
        idx = master_data['TestDistance_m'] == test
        rssi_vals = master_data.loc[idx, 'RSSI'].astype(float).values
        
        # 1) GESD outlier detection: mark outliers as NaN
        outliers = gesd_outliers(rssi_vals, threshold_factor=0.5)
        rssi_vals[outliers] = np.nan
        
        # 2) Interpolate missing values (linear interpolation)
        rssi_series = pd.Series(rssi_vals)
        rssi_interp = rssi_series.interpolate(method='linear').to_numpy()
        
        # 3) Construct variable R array (measurement noise grows with absolute RSSI)
        Rvals = 0.5 + 0.05 * np.abs(rssi_interp)
        
        # 4) Apply the variable-R Kalman filter
        rssi_kf = kalman_filter_1d_variable_R(rssi_interp, Rvals, Q)
        
        # 5) Rolling median smoothing to reduce peaks further
        rssi_final = pd.Series(rssi_kf).rolling(window=window_size, min_periods=1, center=True).median().to_numpy()
        
        # Store the filtered values back in the cleaned_RSSI Series
        cleaned_RSSI.loc[idx] = rssi_final
    
    return cleaned_RSSI